<a href="https://colab.research.google.com/github/abeni-hub/Apex_back/blob/main/NLP_with_RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
%tensorflow_version 2.x
from keras.datasets import imdb
from keras.preprocessing import sequence
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

VOCAB_SIZE = 88584
MAXLEN = 256
BATCH_SIZE = 64
EMBEDDING_DIM = 16
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=VOCAB_SIZE)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [14]:
len(train_data[0])

218

More **Processing

In [15]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

Creating the Model

In [16]:
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(VOCAB_SIZE , 32) ,
  tf.keras.layers.LSTM(32),
  tf.keras.layers.Dense(1,activation ="sigmoid")
])


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2843041 (10.85 MB)
Trainable params: 2843041 (10.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Training

In [18]:
model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["acc"])
history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 68s 105ms/step - loss: 0.4471 - acc: 0.7855 - val_loss: 0.3148 - val_acc: 0.8606
Epoch 2/10
625/625 [==============================] - 64s 102ms/step - loss: 0.2581 - acc: 0.8997 - val_loss: 0.3151 - val_acc: 0.8792
Epoch 3/10
625/625 [==============================] - 64s 103ms/step - loss: 0.2001 - acc: 0.9280 - val_loss: 0.2823 - val_acc: 0.8824
Epoch 4/10
625/625 [==============================] - 67s 107ms/step - loss: 0.1630 - acc: 0.9434 - val_loss: 0.3066 - val_acc: 0.8840
Epoch 5/10
625/625 [==============================] - 68s 109ms/step - loss: 0.1334 - acc: 0.9543 - val_loss: 0.3783 - val_acc: 0.8676
Epoch 6/10
625/625 [==============================] - 64s 103ms/step - loss: 0.1091 - acc: 0.9643 - val_loss: 0.3500 - val_acc: 0.8842
Epoch 7/10
625/625 [==============================] - 67s 107ms/step - loss: 0.0934 - acc: 0.9707 - val_loss: 0.3552 - val_acc: 0.8834
Epoch 8/10
625/625 [==============================] - 6

In [19]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 20s 25ms/step - loss: 0.5299 - acc: 0.8621
[0.5299179553985596, 0.8621199727058411]


Making Predictions

In [25]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
import keras

In [26]:

word_index = imdb.get_word_index()


def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]


text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   

In [27]:
# While were at it lets make a decode functions

reverse_word_index = {value: key for (key, value) in word_index.items()}
def decode_integers(integers):
  PAD = 0
  text = ""
  for num in integers:
    if num != PAD:
      text += reverse_word_index[num] + " "
  return text[:-1]

print(decode_integers(encoded))

that movie was just amazing so amazing


In [28]:
# now time to make a prediction

def prediction(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,256))
  pred[0] = encoded_text
  result = model.predict(pred)
  print(result[0])

positive_review = "That movie was so awesome! I really loved it! and would watch it again because ti was amazingly great"
prediction(positive_review)
negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst thing I've ever watched"
prediction(negative_review)

1/1 [==============================] - 0s 448ms/step
[0.86163664]
1/1 [==============================] - 0s 30ms/step
[0.37309206]
